In [10]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
from django.db.models import F

In [11]:
run_id=359
qid=1603

In [12]:
dts = DocTopic.objects.filter(run_id=run_id).values('topic__title')

dts = dts.annotate(
    ipcc = models.Sum(
        models.Case(
            models.When(doc__ipccref__isnull=False,then=1),default=0, output_field=models.FloatField()
        )
    ),
    no_ipcc = models.Sum(
        models.Case(
            models.When(doc__ipccref__isnull=True,then=1),default=0, output_field=models.FloatField()
        )
    )
)
df =  pd.DataFrame.from_dict(list(dts))

ProgrammingError: column scoping_doc.UT does not exist
LINE 1: ...N "scoping_doc" ON ("tmv_app_doctopic"."doc_id" = "scoping_d...
                                                             ^


In [ ]:
df['total'] = df.ipcc+df.no_ipcc
df['ipcc_pcnt'] = df.ipcc/df.total*100
df = df.sort_values(by='ipcc_pcnt', ascending=0).reset_index(drop=True)
df.head()

In [ ]:
plt.rcParams["figure.figsize"] = [16,10]
bdf = df.set_index(['topic__title'])[['ipcc_pcnt']]
bdf.plot.bar();
plt.xlabel('Topic Title')
plt.savefig('../plots/ipcc_topics_{}.png'.format(run_id),bbox_inches='tight')   
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [16,10]
bdf = df.set_index(['topic__title'])[['ipcc']].sort_values(by='ipcc', ascending=0)
bdf.plot.bar();
plt.xlabel('Topic Title')
plt.savefig('../plots/ipcc_topics_{}.png'.format(run_id),bbox_inches='tight')  
plt.savefig('../plots/ipcc_topics_{}.svg'.format(run_id),bbox_inches='tight')  
plt.show()

In [ ]:
tdocs = Doc.objects.filter(query=qid)
ipdocs = tdocs.filter(ipccref__isnull=False).count()
nipdocs = tdocs.filter(ipccref__isnull=True).count()

ind = [1,2]

fig, ax = plt.subplots()

ax.bar(ind,[ipdocs,nipdocs])
ax.set_xticks(ind)
ax.set_xticklabels(["IPCC Match Found","No IPCC Match Found"])

plt.show()


In [ ]:
ipdocs